In [6]:
from typing import Union, Tuple, Dict, List, Optional
from pathlib import Path

def fetch_mapillary_dataset(
    utility_object_value: str,
    bbox: Union[str, Tuple[float, float, float, float]],
    token: str,
    out_root: Union[str, Path, None] = None,
    class_id: int = 0,
    include_panos: bool = False,
    prefer_2048: bool = True,
    user_agent: str = "mapillary-unified-fetcher/3.4",
    invert_tile_y: bool = True,           # flip Y from tile space → image space (fixes vertical offset)
    max_images: Optional[int] = None,     # cap number of labeled images saved
    max_features: Optional[int] = None,   # cap number of map features processed
    skip_existing: bool = True,           # skip images that already have labels on disk
) -> Dict[str, str]:
    """
    Fetch images + YOLO labels for a Mapillary object (e.g., 'object--fire-hydrant') inside a bbox.

    Returns a dict summary including paths and counts.
    """
    import json, time, base64
    import requests, mapbox_vector_tile as mvt

    try:
        from PIL import Image, ImageOps
        _PIL_OK = True
    except Exception:
        _PIL_OK = False

    assert isinstance(token, str) and token.startswith("MLY|"), "Mapillary token must start with 'MLY|'"
    GRAPH = "https://graph.mapillary.com"

    # ---------- helpers ----------
    def _first_number(x, default=0):
        if x is None:
            return default
        if isinstance(x, (int, float)):
            return x
        if isinstance(x, (list, tuple)):
            for v in x:
                try:
                    return float(v)
                except Exception:
                    continue
            return default
        try:
            return float(x)
        except Exception:
            return default

    def _first_str(x, default=None):
        if x is None:
            return default
        if isinstance(x, str):
            return x
        if isinstance(x, (list, tuple)):
            for v in x:
                if isinstance(v, str):
                    return v
        return default

    # ---------- parse bbox ----------
    if isinstance(bbox, str):
        parts = [float(x.strip()) for x in bbox.split(",")]
        if len(parts) != 4:
            raise ValueError("bbox string must be 'minLon,minLat,maxLon,maxLat'")
        min_lon, min_lat, max_lon, max_lat = parts
    elif isinstance(bbox, (tuple, list)) and len(bbox) == 4:
        min_lon, min_lat, max_lon, max_lat = map(float, bbox)
    else:
        raise ValueError("bbox must be a 4-item tuple/list or a string 'minLon,minLat,maxLon,maxLat'")
    bbox_str = f"{min_lon},{min_lat},{max_lon},{max_lat}"

    # ---------- out dirs ----------
    util_slug = utility_object_value.replace("object--", "").replace("--", "_")
    out_root = Path(out_root or f"out_{util_slug}_det")
    img_dir, lbl_dir, meta_dir = out_root / "images", out_root / "labels", out_root / "meta"
    for d in (img_dir, lbl_dir, meta_dir):
        d.mkdir(parents=True, exist_ok=True)

    # ---------- HTTP ----------
    SESSION = requests.Session()
    SESSION.headers.update({"User-Agent": user_agent})

    def http_get(url, params=None, retry=6, timeout=60):
        params = dict(params or {})
        params["access_token"] = token
        for a in range(retry):
            r = SESSION.get(url, params=params, timeout=timeout)
            if r.status_code == 200:
                try:
                    return r.json()
                except Exception:
                    raise RuntimeError(f"Invalid JSON from {url}")
            transient = r.status_code in (429, 500, 502, 503, 504)
            try:
                err = (r.json() or {}).get("error", {})
                # Some Mapillary throttling codes
                if err.get("code") == -2 or err.get("error_subcode") == 3404014:
                    transient = True
            except Exception:
                pass
            if transient and a < retry - 1:
                time.sleep(1.5 * (a + 1))
                continue
            raise RuntimeError(f"GET {url} -> {r.status_code}: {r.text[:200]}")
        raise RuntimeError("GET failed after retries")

    # ---------- geometry: decode to PIXELS (with optional Y flip), then YOLO ----------
    def decode_polygon_pixels(geometry_b64: str, W: int, H: int):
        vt = base64.b64decode(geometry_b64)
        decoded = mvt.decode(vt)
        layer = next(iter(decoded.values())) if decoded else {}
        extent = float(layer.get("extent", 4096))
        feats = layer.get("features", [])
        if not feats:
            return []
        geom = feats[0].get("geometry", {})
        if geom.get("type") != "Polygon":
            return []
        ring = geom.get("coordinates", [[]])[0]

        pts = []
        if invert_tile_y:
            for x, y in ring:
                pts.append((x / extent * W, (1.0 - y / extent) * H))  # flip Y
        else:
            for x, y in ring:
                pts.append((x / extent * W, (y / extent) * H))
        return pts

    def polygon_to_bbox(pts):
        if not pts:
            return None
        xs, ys = zip(*pts)
        x0, y0, x1, y1 = min(xs), min(ys), max(xs), max(ys)
        return (x0, y0, x1, y1) if (x1 > x0 and y1 > y0) else None

    def rotate_yolo_box(cx, cy, w, h, rot_deg):
        r = int(rot_deg) % 360
        if r == 0:
            return cx, cy, w, h
        if r == 90:
            return cy, 1.0 - cx, h, w
        if r == 180:
            return 1.0 - cx, 1.0 - cy, w, h
        if r == 270:
            return 1.0 - cy, cx, h, w
        return cx, cy, w, h

    # ---------- API wrappers ----------
    def fetch_features(limit=2000):
        data = http_get(
            f"{GRAPH}/map_features",
            params={
                "fields": "id",
                "bbox": bbox_str,
                "object_values": utility_object_value,
                "limit": min(limit, 2000),
            },
        )
        return data.get("data", [])

    def fetch_feature_detections(map_feature_id: str):
        data = http_get(
            f"{GRAPH}/{map_feature_id}/detections",
            params={"fields": "image,value,geometry"},
        )
        return data.get("data", []) if isinstance(data, dict) else (data if isinstance(data, list) else [])

    def get_image_meta(image_id: str):
        fields = ",".join(
            [
                "id",
                "width",
                "height",
                "camera_type",
                "is_pano",
                "exif_orientation",
                "computed_rotation",
                "thumb_original_url",
                "thumb_2048_url",
                "thumb_1024_url",
            ]
        )
        return http_get(f"{GRAPH}/{image_id}", params={"fields": fields})

    def download(url: str, out_path: Path):
        out_path.parent.mkdir(parents=True, exist_ok=True)
        r = SESSION.get(url, timeout=120)
        r.raise_for_status()
        out_path.write_bytes(r.content)

    # =================== PIPELINE ===================
    print(f"Searching '{utility_object_value}' in bbox: {bbox_str}")

    # Determine how many features to request from the API, then clip locally if needed
    request_limit = 2000 if max_features is None else min(max_features, 2000)
    features = fetch_features(limit=request_limit)
    if max_features is not None and len(features) > max_features:
        features = features[:max_features]

    print("Features found (capped):", len(features))
    if not features:
        print("No features returned.")
        return {
            "images_with_labels": 0,
            "out": str(out_root),
            "bbox": bbox_str,
            "utility": utility_object_value,
        }

    # 1) Group detections by image (respecting max_features cap)
    per_image_geoms: Dict[str, List[str]] = {}
    for i, feat in enumerate(features, 1):
        fid = feat.get("id")
        if not fid:
            continue
        try:
            dets = fetch_feature_detections(fid)
        except Exception:
            continue
        for d in dets:
            if d.get("value") != utility_object_value:
                continue
            img = d.get("image") or {}
            image_id = _first_str(img.get("id")) if isinstance(img, dict) else _first_str(img)
            geom_b64 = _first_str(d.get("geometry"))
            if image_id and geom_b64:
                per_image_geoms.setdefault(image_id, []).append(geom_b64)
        if i % 200 == 0:
            print(f"  processed features: {i}/{len(features)}")

    print("Images that contain the object (from detections):", len(per_image_geoms))

    # 2) Per-image processing (still optionally capped by max_images)
    kept = 0
    for image_id, geom_list in per_image_geoms.items():
        if max_images is not None and kept >= max_images:
            break

        # optional: skip work we already did (makes reruns fast)
        if skip_existing and (lbl_dir / f"{image_id}.txt").exists():
            continue

        try:
            meta = get_image_meta(image_id)
        except Exception:
            continue

        cam_type = _first_str(meta.get("camera_type"))
        is_pano = bool(meta.get("is_pano", False))
        if (not include_panos) and (is_pano or cam_type != "perspective"):
            continue

        W = int(_first_number(meta.get("width"), 0))
        H = int(_first_number(meta.get("height"), 0))
        if not (W and H):
            continue

        orig_url = _first_str(meta.get("thumb_original_url"))
        url_2048 = _first_str(meta.get("thumb_2048_url"))
        url_1024 = _first_str(meta.get("thumb_1024_url"))
        url_img = orig_url or (prefer_2048 and url_2048) or url_1024
        if not url_img:
            continue

        rot = _first_number(meta.get("computed_rotation"), 0)
        if rot == 0:
            exif = int(_first_number(meta.get("exif_orientation"), 1))
            rot = {1: 0, 6: 90, 3: 180, 8: 270}.get(exif, 0)
        else:
            rot = int(rot)

        lines = []
        for geom_b64 in geom_list:
            try:
                pts = decode_polygon_pixels(geom_b64, W, H)
            except Exception:
                continue
            bb = polygon_to_bbox(pts)
            if not bb:
                continue
            x0, y0, x1, y1 = bb
            cx = (x0 + x1) / 2 / W
            cy = (y0 + y1) / 2 / H
            w = (x1 - x0) / W
            h = (y1 - y0) / H
            cx, cy, w, h = rotate_yolo_box(cx, cy, w, h, rot)
            cx = max(0.0, min(1.0, cx))
            cy = max(0.0, min(1.0, cy))
            w = max(0.0, min(1.0, w))
            h = max(0.0, min(1.0, h))
            lines.append(f"{class_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}")

        if not lines:
            continue

        # write YOLO label
        (lbl_dir / f"{image_id}.txt").write_text("\n".join(lines), encoding="utf-8")

        # download and upright the image (once)
        p_img = img_dir / f"{image_id}.jpg"
        if not p_img.exists():
            try:
                download(url_img, p_img)
                if _PIL_OK:
                    try:
                        im = Image.open(p_img)
                        im = ImageOps.exif_transpose(im)  # make pixels upright to match rotated labels
                        im.save(p_img)
                    except Exception:
                        pass
            except Exception:
                # ignore download failures; label file already written
                pass

        # write meta
        (meta_dir / f"{image_id}.json").write_text(
            json.dumps(
                {
                    "image_id": image_id,
                    "width": W,
                    "height": H,
                    "camera_type": cam_type,
                    "is_pano": is_pano,
                    "computed_rotation": int(rot),
                    "invert_tile_y": bool(invert_tile_y),
                    "object_value": utility_object_value,
                    "bbox": bbox_str,
                    "num_instances": len(lines),
                },
                indent=2,
            ),
            encoding="utf-8",
        )

        kept += 1

    print(f"Done. Images with labels: {kept}")
    print(f"- Images: {img_dir.resolve()}")
    print(f"- YOLO labels: {lbl_dir.resolve()}")
    return {
        "images_with_labels": kept,
        "out": str(out_root),
        "images_dir": str(img_dir),
        "labels_dir": str(lbl_dir),
        "utility": utility_object_value,
        "bbox": bbox_str,
        "features_used": len(features),
    }


In [7]:

BBOX = (-118.67, 33.70, -118.10, 34.35)


fetch_mapillary_dataset(
     "object--fire-hydrant",
     BBOX,
     token="MLY|9694828473951856|fd2858fda2e17f10b783cd114272679e",
     out_root="out_hydrants_det_val",
     class_id=0,
     max_images=700,      
     max_features=300,
    prefer_2048=False,
 )


Searching 'object--fire-hydrant' in bbox: -118.67,33.7,-118.1,34.35
Features found (capped): 300
  processed features: 200/300
Images that contain the object (from detections): 1671
Done. Images with labels: 700
- Images: C:\Users\GLazar\Downloads\out_hydrants_det_val\images
- YOLO labels: C:\Users\GLazar\Downloads\out_hydrants_det_val\labels


{'images_with_labels': 700,
 'out': 'out_hydrants_det_val',
 'images_dir': 'out_hydrants_det_val\\images',
 'labels_dir': 'out_hydrants_det_val\\labels',
 'utility': 'object--fire-hydrant',
 'bbox': '-118.67,33.7,-118.1,34.35',
 'features_used': 300}

In [8]:

fetch_mapillary_dataset(
     "object--junction-box",
     BBOX,
     token="MLY|9694828473951856|fd2858fda2e17f10b783cd114272679e",
     out_root="out_junction_det_val",
     class_id=1,
    max_images=700, 
    max_features=200,
    prefer_2048=False,     # faster downloads
 )


Searching 'object--junction-box' in bbox: -118.67,33.7,-118.1,34.35
Features found (capped): 200
  processed features: 200/200
Images that contain the object (from detections): 1127
Done. Images with labels: 585
- Images: C:\Users\GLazar\Downloads\out_junction_det_val\images
- YOLO labels: C:\Users\GLazar\Downloads\out_junction_det_val\labels


{'images_with_labels': 585,
 'out': 'out_junction_det_val',
 'images_dir': 'out_junction_det_val\\images',
 'labels_dir': 'out_junction_det_val\\labels',
 'utility': 'object--junction-box',
 'bbox': '-118.67,33.7,-118.1,34.35',
 'features_used': 200}

In [13]:
fetch_mapillary_dataset(
     "object--manhole",
     BBOX,
     token="MLY|9694828473951856|fd2858fda2e17f10b783cd114272679e",
     out_root="out_manhole_det_val",
     class_id=2,
     max_images=700, 
     max_features=400,
     prefer_2048=False,     # faster downloads
 )

Searching 'object--manhole' in bbox: -118.67,33.7,-118.1,34.35
Features found (capped): 400
  processed features: 200/400
  processed features: 400/400
Images that contain the object (from detections): 1712
Done. Images with labels: 700
- Images: C:\Users\GLazar\Downloads\out_manhole_det_val\images
- YOLO labels: C:\Users\GLazar\Downloads\out_manhole_det_val\labels


{'images_with_labels': 700,
 'out': 'out_manhole_det_val',
 'images_dir': 'out_manhole_det_val\\images',
 'labels_dir': 'out_manhole_det_val\\labels',
 'utility': 'object--manhole',
 'bbox': '-118.67,33.7,-118.1,34.35',
 'features_used': 400}

In [10]:

fetch_mapillary_dataset(
     "object--parking-meter",
     BBOX,
     token="MLY|9694828473951856|fd2858fda2e17f10b783cd114272679e",
     out_root="out_parking_det_val",
     class_id=3,
     max_images=700,           
    max_features=200,
    prefer_2048=False,     # faster downloads
 )

Searching 'object--parking-meter' in bbox: -118.67,33.7,-118.1,34.35
Features found (capped): 200
  processed features: 200/200
Images that contain the object (from detections): 746
Done. Images with labels: 524
- Images: C:\Users\GLazar\Downloads\out_parking_det_val\images
- YOLO labels: C:\Users\GLazar\Downloads\out_parking_det_val\labels


{'images_with_labels': 524,
 'out': 'out_parking_det_val',
 'images_dir': 'out_parking_det_val\\images',
 'labels_dir': 'out_parking_det_val\\labels',
 'utility': 'object--parking-meter',
 'bbox': '-118.67,33.7,-118.1,34.35',
 'features_used': 200}

In [20]:
fetch_mapillary_dataset(
     "object--water-valve",
     BBOX,
     token="MLY|9694828473951856|fd2858fda2e17f10b783cd114272679e",
     out_root="out_water_det_val",
     class_id=4,
     max_images=700,     
    max_features=100,
    prefer_2048=False,     # faster downloads
 )

Searching 'object--water-valve' in bbox: -118.67,33.7,-118.1,34.35
Features found (capped): 100
Images that contain the object (from detections): 355
Done. Images with labels: 0
- Images: C:\Users\GLazar\Downloads\out_water_det_val\images
- YOLO labels: C:\Users\GLazar\Downloads\out_water_det_val\labels


{'images_with_labels': 0,
 'out': 'out_water_det_val',
 'images_dir': 'out_water_det_val\\images',
 'labels_dir': 'out_water_det_val\\labels',
 'utility': 'object--water-valve',
 'bbox': '-118.67,33.7,-118.1,34.35',
 'features_used': 100}

In [21]:
import os, shutil
from pathlib import Path

# Paths
PROJECT_ROOT = Path(r"D:\Utilities Project")  # <-- change to your real project folder
EXTRA_ROOT   = Path(r"C:\Users\GLazar\Downloads\Additional Utilities\val")  # folder containing the 5 utility folders

DEST_IMAGES = PROJECT_ROOT / "images" / "val"
DEST_LABELS = PROJECT_ROOT / "labels" / "val"

# make sure destinations exist
DEST_IMAGES.mkdir(parents=True, exist_ok=True)
DEST_LABELS.mkdir(parents=True, exist_ok=True)

# walk through each utility subfolder
for util_folder in EXTRA_ROOT.iterdir():
    if not util_folder.is_dir():
        continue
    
    images_dir = util_folder / "images"
    labels_dir = util_folder / "labels"
    
    # copy all images
    if images_dir.exists():
        for img in images_dir.rglob("*.*"):
            shutil.move(str(img), DEST_IMAGES / img.name)
    
    # copy all labels
    if labels_dir.exists():
        for lbl in labels_dir.rglob("*.txt"):
            shutil.move(str(lbl), DEST_LABELS / lbl.name)

print("✅ All utility images moved to", DEST_IMAGES)
print("✅ All utility labels moved to", DEST_LABELS)


✅ All utility images moved to D:\Utilities Project\images\val
✅ All utility labels moved to D:\Utilities Project\labels\val
